In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install sentencepiece

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from evaluate import load

dataset  = load_dataset("toanduc/paper-gpt")

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
from transformers import AutoTokenizer, FlaxMBartForConditionalGeneration, MBartConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer,AutoModel

# Load pre-trained BART model and tokenizer
model_name = "vinai/bartpho-syllable"
tokenizer = BartForConditionalGeneration.from_pretrained(model_name)
model = AutoTokenizer.from_pretrained(model_name)

You are using a model of type mbart to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.


In [ ]:
max_input_length = 1024
max_target_length = 128

def process_dataset(examples):
    return {"article": examples["Original Text"], "highlight": examples["Summary"]}

tokenized_datasets = dataset.map(process_dataset, batched=True)

In [ ]:
# Define data collator
def collate_fn(batch):
    return tokenizer.pad(batch, return_tensors="pt")

In [ ]:
# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./bart_finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=4,
    logging_dir="./logs",
    save_total_limit=3,
    learning_rate=2e-5,
)

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
    data_collator=collate_fn,
)

AttributeError: ignored

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")

# Optionally, upload the fine-tuned model to the Hugging Face Model Hub
trainer.push_to_hub()

In [ ]:


# Tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["article"], truncation=True, max_length=1024, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define data collator
def collate_fn(batch):
    return tokenizer.pad(batch, return_tensors="pt")

# Fine-tuning arguments
training_args = TrainingArguments(
    output_dir="./bart_finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2,
    push_to_hub=False,
    learning_rate=3e-5,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=collate_fn,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./bart_finetuned")
